In [55]:
import os, json
import pandas as pd
import numpy as np
from PIL import Image
import cv2 as cv
import matplotlib.pyplot as plt
import math
from matplotlib import cm


In [56]:
image_list = os.listdir(r"C:\Users\adama\OneDrive\Dokument\marginalia\png")
image_path = r"C:\Users\adama\OneDrive\Dokument\marginalia\png"
df = pd.read_csv(r"C:\Users\adama\OneDrive\Dokument\marginalia\data.csv")
df.head()

,number,jsonname,photoname,width,height,class,xmin,ymin,xmax,ymax
0,189279,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3343,4157,m,633.645161,3258.064516,2585.258065,3745.161290
1,184298,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3678,4806,m,133.594595,1810.810811,2825.486486,2072.972973
2,184332,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3582,4776,m,2647.000000,194.000000,2873.315789,399.263158
3,184332,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3582,4776,m,294.368421,3351.894737,452.263158,3562.421053
4,184332,/Users/imchengliang/Downloads/DS-Project/json-...,/Users/imchengliang/Downloads/DS-Project/photo...,3582,4776,m,1036.473684,3257.157895,1220.684211,3467.684211


In [57]:

def png_to_np(path):
    """
    Input: .png file
    Output: np.array
    """
    # read image
    img = cv.imread(path)
    # convert to grayscale 
    gray_scale_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    return gray_scale_img

In [58]:
#reshape
def down_scale_img(img):
    new_width  = 500
    new_height = 350


    resized_image = cv.resize(img, (new_width, new_height))
    return resized_image

In [59]:
import warnings
warnings.filterwarnings('ignore')

#scale down images as well as make them black/white
new_df = pd.DataFrame()
counter = 0
for img in image_list:
    
    black_white = png_to_np(image_path + "/" + img)
    old_size = black_white.shape
    #print(old_size)
    ratio_x = (500 / old_size[1])
    ratio_y = (350 / old_size[0])
    #print(ratio_y)
    #get the coordinates of the old points
    rows = df.loc[df['number'] == int(img[:-4])]
    #new columns for reshaped coordinates
    rows["xmin_scaled"] = rows["xmin"].apply(lambda x: x*ratio_x)
    rows["xmax_scaled"] = rows["xmax"].apply(lambda x: x*ratio_x)
    rows["ymin_scaled"] = rows["ymin"].apply(lambda x: x*ratio_y)
    rows["ymax_scaled"] = rows["ymax"].apply(lambda x: x*ratio_y)
    new_df = pd.concat([new_df, rows])
    #print(new_df.loc[df['number'] == int(img[:-4])])
    down_scaled = down_scale_img(black_white)
    im = Image.fromarray(down_scaled)
    im.save(r"C:\Users\adama\OneDrive\Dokument\marginalia\rescaled_images" + "/" + str(img))
    counter += 1
    print(img)
    if counter % 10 == 0:
        print("Progress: " + str(round(counter/len(image_list)*100, 2)) + "%")
        




183786.png
183844.png
183845.png
183846.png
183847.png
183855.png
183862.png
183867.png
183868.png
183869.png
Progress: 2.0%
183874.png
183876.png
183878.png
183881.png
183919.png
183942.png
183943.png
183971.png
184010.png
184011.png
Progress: 3.99%
184012.png
184015.png
184018.png
184064.png
184065.png
184067.png
184068.png
184076.png
184078.png
184101.png
Progress: 5.99%
184102.png
184104.png
184105.png
184127.png
184130.png
184145.png
184146.png
184148.png
184156.png
184157.png
Progress: 7.98%
184159.png
184251.png
184252.png
184253.png
184255.png
184257.png
184258.png
184259.png
184260.png
184261.png
Progress: 9.98%
184262.png
184263.png
184266.png
184267.png
184269.png
184270.png
184283.png
184284.png
184293.png
184294.png
Progress: 11.98%
184296.png
184298.png
184301.png
184305.png
184307.png
184310.png
184322.png
184328.png
184331.png
184332.png
Progress: 13.97%
184334.png
184335.png
184337.png
184338.png
184400.png
184404.png
184406.png
184407.png
184409.png
184410.png
Progres

In [60]:
print(new_df.head())


      number                                           jsonname  \
1510  183786  /Users/imchengliang/Downloads/DS-Project/json-...   
1511  183786  /Users/imchengliang/Downloads/DS-Project/json-...   
1512  183786  /Users/imchengliang/Downloads/DS-Project/json-...   
726   183844  /Users/imchengliang/Downloads/DS-Project/json-...   
442   183845  /Users/imchengliang/Downloads/DS-Project/json-...   

                                              photoname  width  height class  \
1510  /Users/imchengliang/Downloads/DS-Project/photo...   2272    3574     m   
1511  /Users/imchengliang/Downloads/DS-Project/photo...   2272    3574     m   
1512  /Users/imchengliang/Downloads/DS-Project/photo...   2272    3574     m   
726   /Users/imchengliang/Downloads/DS-Project/photo...   3424    4740     m   
442   /Users/imchengliang/Downloads/DS-Project/photo...   3352    4693     m   

             xmin         ymin         xmax         ymax  xmin_scaled  \
1510     4.000000   130.000000   141.500000

In [61]:
#save new df to csv
new_df.to_csv(r"C:\Users\adama\OneDrive\Dokument\marginalia\rescaled_data.csv")

In [64]:
#see if boxes are correct
image = cv.imread(r"C:\Users\adama\OneDrive\Dokument\marginalia\rescaled_images\183786.png")
window_name = 'Image'
color = (255, 0, 0)
start_point = (int(new_df.iloc[2]["xmin_scaled"]), int(new_df.iloc[2]["ymin_scaled"]))
end_point = (int(new_df.iloc[2]["xmax_scaled"]), int(new_df.iloc[2]["ymax_scaled"]))
thickness = 2

image = cv.rectangle(image, start_point, end_point, color, thickness)
cv.imshow(window_name, image) 
cv.waitKey(0)
cv.destroyAllWindows()
